# CS 594 Project Code Analysis
<h3>Hemanth Srinivas Reddy Chennur</h3> 
<h3>Adithya Reddy Chidirala</h3>
<h3>Yashwanth Reddy Dasari Reddy</h3>

<h4>Problem 1: [25 points]</h4>
Implement the quad-tree (not the compressed one) we have seen in class for the approximat 
nearest neighbor queries. Your quad-tree should work in any dimension. Let Dquad be t  
quad tree data structure.

In Dataset you will find a set of points (please copy paste it into a txt file in your mach e). 
Your code should read this file line by line containing a set P of n points. The firs line 
contains the dimension d, and every other line contains a tuple x₁, x₂, ..., xd that reprsents 
the point x ∈ Rᵈ with coordinates x₁, ..., xd. Your implementation should const ct the 
quadtree for ANN queries (as we have seen in class) over the set of points P. Reort the 
construction time. What is the spread of the point set? What is the height of he tree? 
Describe the relation between the spread and the height of the tree. of the tree.

In [1]:
#Question 1
import numpy as np
import time

class QuadNode:
    def __init__(self, center, width):
        self.center = center
        self.width = width
        self.point = None
        self.children = None

    def is_leaf(self):
        return self.children is None



In [2]:
class QuadTree:
    def __init__(self, points):
        self.points = points
        self.dim = points.shape[1]

        self.min_coords = np.min(points, axis=0)
        self.max_coords = np.max(points, axis=0)

        self.center = (self.min_coords + self.max_coords) / 2
        self.width = np.max(self.max_coords - self.min_coords) / 2

        start = time.time()
        self.root = self.build_tree(points, self.center, self.width)
        self.build_time = time.time() - start

        self.height = self.get_height(self.root)
        self.spread = self.get_spread()

    def build_tree(self, points, center, width):
        node = QuadNode(center, width)

        if len(points) == 0:
            return node
        if len(points) == 1:
            node.point = points[0]
            return node

        node.children = []
        for i in range(2 ** self.dim):
            child_center = center.copy()
            for d in range(self.dim):
                child_center[d] += width/2 if i & (1 << d) else -width/2

            mask = np.ones(len(points), dtype=bool)
            for d in range(self.dim):
                mask &= points[:, d] >= center[d] if i & (1 << d) else points[:, d] < center[d]

            child_points = points[mask]
            node.children.append(self.build_tree(child_points, child_center, width/2))

        return node

    def get_height(self, node):
        if node.is_leaf():
            return 0
        return 1 + max(self.get_height(child) for child in node.children)

    def get_spread(self):
        min_dist = float('inf')
        max_dist = 0

        for i in range(len(self.points)):
            for j in range(i+1, len(self.points)):
                dist = np.linalg.norm(self.points[i] - self.points[j])
                min_dist = min(min_dist, dist)
                max_dist = max(max_dist, dist)

        return max_dist / min_dist if min_dist > 0 else float('inf')



In [6]:
def read_points(filename):
    points = []
    with open(filename, encoding='utf-8-sig') as f:  # utf-8-sig handles the BOM character
        for line in f:
            coords = [float(x) for x in line.strip().split(',')]
            points.append(coords)
    return np.array(points)

In [7]:
points = read_points('Pquad.txt')
tree = QuadTree(points)

print(f"Time to build tree: {tree.build_time:.3f} seconds")
print(f"Height of tree: {tree.height}")
print(f"Spread of points: {tree.spread:.2f}")

Time to build tree: 0.505 seconds
Height of tree: 13
Spread of points: 15550.61
